In [ ]:
!pwd

/content


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git



  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-3_k64i3p
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-3_k64i3p
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=d850eac3db11af1c1825016028decab4db65db2ed3d91a78521c579da1c0f862
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezikw0fc/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name prg1.cu
#include <stdio.h>
#include <stdlib.h>
#include <math.h>
#include "cuda_runtime.h"

__global__ void convolutionKernel(int* N, int* M, int* P, int width, int mask_width) {
    int pval = 0;
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    int start = i - (mask_width / 2);
    for (int j = 0; j < mask_width; j++) {
        if (start >= 0 && start < width) {
            pval += N[start + j] * M[j];
        }
    }
    P[i] = pval;
}

void performConvolution(int* N, int* M, int* P, int width, int mask_width) {
    int *d_N, *d_M, *d_P;
    int size = width * sizeof(int);

    cudaMalloc((void**)&d_N, size);
    cudaMalloc((void**)&d_M, mask_width * sizeof(int));
    cudaMalloc((void**)&d_P, size);

    cudaMemcpy(d_N, N, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_M, M, mask_width * sizeof(int), cudaMemcpyHostToDevice);

    int gridSize = (width + 255) / 256;  // Assuming 256 threads per block
    int blockSize = 256;

    convolutionKernel<<<gridSize, blockSize>>>(d_N, d_M, d_P, width, mask_width);

    cudaMemcpy(P, d_P, size, cudaMemcpyDeviceToHost);

    cudaFree(d_N);
    cudaFree(d_M);
    cudaFree(d_P);
}

int main() {
    int width, mask_width;
    printf("Enter the width:\n");
    scanf("%d", &width);
    printf("Enter the mask width of the array:\n");
    scanf("%d", &mask_width);

    int* N = (int*)malloc(sizeof(int) * width);
    int* M = (int*)malloc(sizeof(int) * mask_width);
    int* P = (int*)malloc(sizeof(int) * width);

    printf("Enter the elements in the array:\n");
    for (int i = 0; i < width; i++)
        scanf("%d", &N[i]);
    printf("Enter the elements in the mask:\n");
    for (int i = 0; i < mask_width; i++)
        scanf("%d", &M[i]);

    performConvolution(N, M, P, width, mask_width);

    printf("Result:\n");
    for (int i = 0; i < width; i++) {
        printf("%d ", P[i]);
    }

    free(N);
    free(M);
    free(P);

    return 0;
}


'File written in /content/src/prg1.cu'

In [ ]:
!nvcc /content/src/prg1.cu -o /content/src/prg1

In [ ]:
!/content/src/prg1

Enter the width:
1
Enter the mask width of the array:
1
Enter the elements in the array:
1
Enter the elements in the mask:
1
Result:
1 

In [ ]:
%%cuda --name prg2.cu
#include<cuda_runtime.h>
#include<stdio.h>
#include<stdlib.h>
__global__ void rgbtogray(unsigned char* grayImg,unsigned char* rgbImg,int height,int width){
    int col = threadIdx.x + blockDim.x*blockIdx.x;
    int row = threadIdx.y + blockDim.y*blockIdx.y;
    if (row < height && col < width){
        int grayOffset = row*width + col;
        int rbgOffset = grayOffset*3;
        unsigned char r = rgbImg[rbgOffset];
        unsigned char g = rgbImg[rbgOffset + 1];
        unsigned char b = rgbImg[rbgOffset + 2];
        grayImg[grayOffset] = 0.21f * r + 0.72f * g + 0.07f * b;
    }
}

void RgbtoGray(unsigned char* grayImg,unsigned char* rgbImg,int height,int width){
    int rgb_size = height*width*3*sizeof(unsigned char);
    int gray_size = height*width*sizeof(unsigned char);
    unsigned char* d_rgb, *d_gray;
    for (int i = 0; i < width*height*3; i++)
        rgbImg[i] = (i % 256);

    printf("Original values:\n");
    for (int i = 0; i < height; i++){
        for (int j = 0; j < width; j++){
            int idx = i*width + j;
            printf("(%u %u %u) ", rgbImg[idx], rgbImg[idx + 1],rgbImg[idx + 2]);
        }
        printf("\n");
    }

    cudaMalloc((void**)&d_gray,gray_size);
    cudaMalloc((void**)&d_rgb,rgb_size);
    cudaMemcpy(d_rgb,rgbImg,rgb_size,cudaMemcpyHostToDevice);

    dim3 blockSize(32,32);
    dim3 gridSize((width + blockSize.x - 1)/blockSize.x,(height + blockSize.y - 1)/blockSize.y);
    rgbtogray<<<gridSize,blockSize>>>(d_gray,d_rgb,height,width);
    cudaMemcpy(grayImg,d_gray,gray_size,cudaMemcpyDeviceToHost);
    printf("Converted grayscale values:\n");
    for (int i = 0; i < height; i++){
        for (int j = 0; j < width; j++){
            int idx = i*width + j;
            printf("%u ",grayImg[idx]);
        }
        printf("\n");
    }
}

int main(){
    int height = 3, width = 3;
    unsigned char* gray = (unsigned char*)malloc(height*width*sizeof(unsigned char));
    unsigned char* rgb = (unsigned char*)malloc(height*width*3*sizeof(unsigned char));
    RgbtoGray(gray,rgb,height,width);
    return 0;
}


'File written in /content/src/prg2.cu'

In [ ]:
!nvcc /content/src/prg2.cu -o /content/src/prg2

In [ ]:
!/content/src/prg2

Original values:
(0 1 2) (1 2 3) (2 3 4) 
(3 4 5) (4 5 6) (5 6 7) 
(6 7 8) (7 8 9) (8 9 10) 
Converted grayscale values:
0 3 6 
9 12 15 
18 21 24 


In [ ]:
%%cuda --name prg3.cu

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

// Define the kernel to perform matrix multiplication row-wise
__global__ void multiply_rowWise(int* a, int* b, int* c, int wa, int wb) {
    int ridA = threadIdx.x;
    int sum;

    for (int cidB = 0; cidB < wb; cidB++) {
        sum = 0;

        for (int k = 0; k < wa; k++) {
            sum += (a[ridA * wa + k] * b[k * wb + cidB]);
        }

        c[ridA * wb + cidB] = sum;
    }
}

// Define the kernel to perform matrix multiplication column-wise
__global__ void multiply_Colwise(int* a, int* b, int* c, int ha, int wa) {
    int cidB = threadIdx.x;
    int wb = blockDim.x; // Corrected variable declaration

    for (int ridA = 0; ridA < ha; ridA++) {
        int sum = 0; // Initialize 'sum' to 0 for each column

        for (int k = 0; k < wa; k++) {
            sum += (a[ridA * wa + k] * b[k * wb + cidB]); // Perform element-wise multiplication and accumulate the result
        }

        c[ridA * wb + cidB] = sum; // Store the accumulated 'sum' in the corresponding position of matrix 'c'
    }
}

// Define the kernel to perform matrix multiplication
__global__ void multiplyKernel(int* a, int* b, int* c, int wa, int wb) {
    int ridA = threadIdx.y; // Use threadIdx.y for rows
    int cidB = threadIdx.x; // Use threadIdx.x for columns

    int sum = 0;

    for (int k = 0; k < wa; k++) {
        sum += (a[ridA * wa + k] * b[k * wb + cidB]);
    }

    c[ridA * wb + cidB] = sum;
}

// Function to perform matrix multiplication
void Multiply(int* a, int* b, int* c, int wa, int wb, int ha) {
    int* d_A, * d_B, * d_C; // Declare device memory pointers

    // Calculate the required memory size
    int size = ha * wb * sizeof(int);

    // Allocate device memory
    cudaMalloc((void**)&d_A, size);
    cudaMalloc((void**)&d_B, size);
    cudaMalloc((void**)&d_C, size);

    // Copy data from host to device
    cudaMemcpy(d_A, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_B, b, size, cudaMemcpyHostToDevice);

    // Launch the kernel to perform matrix multiplication row-wise
    multiply_rowWise<<<1, ha>>>(d_A, d_B, d_C, wa, wb);

    // Copy the result from device to host
    cudaMemcpy(c, d_C, size, cudaMemcpyDeviceToHost);

    printf("Result of row-wise matrix multiplication:\n");
    for (int i = 0; i < ha; i++) {
        for (int j = 0; j < wb; j++) {
            printf("%d ", c[i * wb + j]);
        }
        printf("\n");
    }

    // Launch the kernel to perform matrix multiplication column-wise
    multiply_Colwise<<<1, wb>>>(d_A, d_B, d_C, ha, wa);

    // Copy the result from device to host
    cudaMemcpy(c, d_C, size, cudaMemcpyDeviceToHost);

    printf("\nResult of column-wise matrix multiplication:\n");
    for (int i = 0; i < ha; i++) {
        for (int j = 0; j < wb; j++) {
            printf("%d ", c[i * wb + j]);
        }
        printf("\n");
    }

    // Launch the kernel to perform matrix multiplication
    dim3 blockSize(wb, ha);
    multiplyKernel<<<1, blockSize>>>(d_A, d_B, d_C, wa, wb);

    // Copy the result from device to host
    cudaMemcpy(c, d_C, size, cudaMemcpyDeviceToHost);

    printf("\nResult of matrix multiplication element-wise:\n");
    for (int i = 0; i < ha; i++) {
        for (int j = 0; j < wb; j++) {
            printf("%d ", c[i * wb + j]);
        }
        printf("\n");
    }

    // Free device memory
    cudaFree(d_A);
    cudaFree(d_B);
    cudaFree(d_C);
}

int main() {
    int* a, * b, * c;
    int wa, ha, wb, hb;

    printf("Enter the dimensions of matrix A (ha wa):\n");
    scanf("%d %d", &ha, &wa);
    printf("Enter the dimensions of matrix B (hb wb):\n");
    scanf("%d %d", &hb, &wb);

    // Check if matrix dimensions are compatible for multiplication
    if (wa != hb) {
        printf("Matrix dimensions are not compatible for multiplication.\n");
        return 1;
    }

    // Allocate memory for matrices a, b, and c
    a = (int*)malloc(ha * wa * sizeof(int));
    b = (int*)malloc(hb * wb * sizeof(int));
    c = (int*)malloc(ha * wb * sizeof(int));

    printf("Enter the elements of matrix A:\n");
    for (int i = 0; i < ha; i++) {
        for (int j = 0; j < wa; j++) {
            scanf("%d", &a[i * wa + j]);
        }
    }

    printf("Enter the elements of matrix B:\n");
    for (int i = 0; i < hb; i++) {
        for (int j = 0; j < wb; j++) {
            scanf("%d", &b[i * wb + j]);
        }
    }

    // Call the Multiply function to perform matrix multiplication
    Multiply(a, b, c, wa, wb, ha);

    // Free host memory
    free(a);
    free(b);
    free(c);

    return 0;
}


'File written in /content/src/prg3.cu'

In [ ]:
!nvcc /content/src/prg3.cu -o /content/src/prg3

In [ ]:
!/content/src/prg3

Enter the dimensions of matrix A (ha wa):
3 3
Enter the dimensions of matrix B (hb wb):
3 3
Enter the elements of matrix A:
1 2 3
4 5 6
7 8 9
Enter the elements of matrix B:
1 2 3 
4 5 6
7 8 9
Result of row-wise matrix multiplication:
30 36 42 
66 81 96 
102 126 150 

Result of column-wise matrix multiplication:
30 36 42 
66 81 96 
102 126 150 

Result of matrix multiplication element-wise:
30 36 42 
66 81 96 
102 126 150 
